In [2]:
#-----------------------------
'''
Name: Work Injuries Tool
Created on 12/04/2019
Created by Meric Birol
'''
#-----------------------------
#import modules
import os
import arcpy
os.chdir(r'C:\GISProgramming\project')
# Set environment settings

arcpy.env.workspace = 'C:\GISProgramming\project\project.gdb'
arcpy.env.overwriteOutput = True


#setting local variables
#setting parameters
    
excelTable =  "C:\GISProgramming\ExampleExcel_2.xlsx"#arcpy.GetParameterAsText(0)
mainFC =  'FC_Example'  #arcpy.GetParameterAsText(1)
outputGDB = 'C:\GISProgramming\project\project.gdb' # arcpy.GetParameterAsText(2)

outputTable = 'injuries'
outputTableNew = 'injuries_new'
newFeaturesTable = 'new_injuries'
geocodedTableNew = 'geocoded_injuries_new'
copiedGeocodedTableNew= 'geocoded_injuries_new_copy'
HCFacilitiesLocator = r"C:\GISProgramming\project\HCFacilitiesLocator.loc"


#-----------------------------
#create functions
def field_name_list(inputTable):
    '''
    This function takes one argument (for example, feature class or a dbf table) 
    and returns a list of field names.
    '''
    fieldNameList = [f.name.upper() for f in arcpy.ListFields(inputTable)]
    return fieldNameList

def check_field_names(testTable,baseTable):
    ''' 
    This function returns field names which were different between 
    the new excel file (db table) and main feature class. 
    The function takes 2 arguments; testTable and baseTable.
    '''
    testTableFields=field_name_list(testTable)
    baseTableFields=field_name_list(baseTable)

    unmatched_list = [i for i in baseTableFields if not i in testTableFields if not 'SHAPE' in i]
    unmatched_list2 = [i for i in testTableFields if not i in baseTableFields]

    return (unmatched_list + unmatched_list2)



#defines a function that stores the names of the fields which has the substring in the Field Map
def field_map_keyword(inputTable,subString):
    '''
    This function takes two arguments, a feature class and a sub string as string,
    This function creates field mapping objects of a file with field names which has substring text.     
    '''
    fieldMapping = [i for i in field_name_list(inputTable) if subString in i]
    fMap = arcpy.FieldMappings()
    for f in fieldMapping:
        fieldMap = arcpy.FieldMap()
        fieldMap.addInputField(inputTable,f)
        fMap.addFieldMap(fieldMap)
    return fMap
#-----------------------------

In [2]:
try:
    #create database table from the excel sheet
    arcpy.ExcelToTable_conversion(excelTable, outputTableNew, "Sheet1")
    print(arcpy.GetMessages())
except:
    print(arcpy.GetMessages())    
    

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

Start Time: Thursday, December 5, 2019 5:00:17 PM
Running script Excel To Table...
Completed script Excel To Table...
Succeeded at Thursday, December 5, 2019 5:00:21 PM (Elapsed Time: 4.74 seconds)


In [ ]:
func = check_field_names(outputTableNew,mainFC)
if len(func)!=0:
    raise ValueError('The excel table names and the Main feature class names do not match')

In [3]:
def update_existing_features(mainFeatureClass, newTable, fieldUpdated):
    '''
    Updates existing feature class fields with new table's fields.
    This function takes 3 arguments:
    argument 1: the feature class name to update
    argument 2: the new table to get the updates
    argument 3: which field to update
    '''
    try:
        fieldName = "claim_number"

        sql = "{}.{} IS NULL".format(mainFeatureClass, fieldName)
        existingField = '{}.{}'.format(mainFeatureClass,fieldUpdated)
        newField = '{}.{}'.format(newTable,fieldUpdated)
        expression = '!' + newField + '!'
        
        joinTable=arcpy.AddJoin_management(mainFeatureClass, fieldName, 
                                           outputTableNew, fieldName,
                                           'KEEP_ALL')
        selectedFeatures = arcpy.SelectLayerByAttribute_management(joinTable, 
                                                                   "NEW_SELECTION",sql)
        arcpy.CalculateField_management(selectedFeatures, 
                                        existingField, 
                                        expression,
                                        "PYTHON3")
        arcpy.SelectLayerByAttribute_management(joinTable, "CLEAR_SELECTION")
        arcpy.RemoveJoin_management (joinTable,newTable)
        print(arcpy.GetMessages())
    except:
        print(arcpy.GetMessages())
update_existing_features(mainFC,outputTableNew,'CASE_STATUS')

Start Time: Thursday, December 12, 2019 8:40:52 AM
Failed to execute. Parameters are not valid.
WARNING 000970: The join field CLAIM_NUMBER in the join table FC_Example is not indexed. To improve performance, we recommend that an index be created for the join field in the join table.
ERROR 000732: Join Table: Dataset injuries_new does not exist or is not supported
Failed to execute (AddJoin).
Failed at Thursday, December 12, 2019 8:40:52 AM (Elapsed Time: 0.20 seconds)


In [1]:
def export_new_records(newTable, mainFeatureClass, outputName,fieldName):
    '''
    exports new records from the new table
    this function takes 3 arguments:
    argument 1: the new table to export features from
    argument 2: the existing feature class 
    argument 3: output table name
    '''
    try:
        fieldName = 'CLAIM_NUMBER'
        sql = "{}.{} IS NULL".format(mainFeatureClass, fieldName)
        joinTable=arcpy.AddJoin_management(newTable, fieldName,
                                           mainFeatureClass, fieldName,
                                           'KEEP_ALL')
        selectedFeatures = arcpy.SelectLayerByAttribute_management(joinTable, "NEW_SELECTION",sql)
        arcpy.RemoveJoin_management (joinTable,mainFeatureClass)
        arcpy.TableToTable_conversion(selectedFeatures,outputGDB, outputName)
        arcpy.SelectLayerByAttribute_management(joinTabe, "CLEAR_SELECTION")
        print(arcpy.GetMessages())
    except:
        print(arcpy.GetMessages())
        
export_new_records(outputTableNew, mainFC, newFeaturesTable,'CLAIM_NUMBER')

NameError: name 'outputTableNew' is not defined

In [8]:
try:

    #geocoding the table with previously created locator
    arcpy.GeocodeAddresses_geocoding(in_table=newFeaturesTable, 
                                     address_locator=HCFacilitiesLocator, 
                                     in_address_fields="'Single Line Input' Facility_ID VISIBLE NONE",
                                     out_feature_class=geocodedTableNew, 
                                     out_relationship_type="STATIC", 
                                     country="",
                                     location_type="ROUTING_LOCATION", 
                                     category="")

    # locating unmatched records to the beebe lake
    # the given coordinates below is a point in the lake
    fields = ["Status", "SHAPE@XY"]
    cursor = arcpy.da.UpdateCursor(geocodedTableNew, fields )
    xy = (441615.589230, 5002476.246096)
    for row in cursor:
        if row[0] == "U":
            row[1]=xy
        cursor.updateRow(row)
    print('Locating to the Bebee lake is completed succesfully')

    FieldMap = field_map_keyword(geocodedTableNew,'USER_')
    #ArcGIS's feature class to feature class tool will be performed and the new feature class
    #has only the fields which have the USER_ prefix

    arcpy.FeatureClassToFeatureClass_conversion(in_features=geocodedTableNew, 
                                                out_path=outputGDB, 
                                                out_name=copiedGeocodedTableNew,
                                                where_clause="", 
                                                field_mapping=FieldMap)


    #alter field names - delete "user_" prefix
    subs = 'USER_'
    for i in field_name_list(copiedGeocodedTableNew):
        if subs in i:
            arcpy.AlterField_management(in_table=copiedGeocodedTableNew,
                                        field=i,
                                        new_field_name=i[5:])

    #append the data to the original feature class    
    fieldMapping = []
    for i in field_name_list(copiedGeocodedTableNew):
        if not i=='OBJECTID':
            if not i=='SHAPE':
                fieldMapping.append(i)
    fMap = arcpy.FieldMappings()
    # for each field, create an individual field map, and add it to the field mapping object
    for field in fieldMapping:
        fieldMap = arcpy.FieldMap()
        fieldMap.addInputField(copiedGeocodedTableNew, field)
        fMap.addFieldMap(fieldMap)

    arcpy.Append_management(inputs=copiedGeocodedTableNew, 
                            target=mainFC, 
                            schema_type="NO_TEST", 
                            field_mapping=fMap)
except:
    print(arcpy.GetMessages())


Locating to the Bebee lake is completed succesfully
Start Time: Thursday, December 5, 2019 5:03:53 PM
Succeeded at Thursday, December 5, 2019 5:03:53 PM (Elapsed Time: 0.37 seconds)


In [ ]:
finally:
    try:
        i = [outputTable,outputTableNew,newFeaturesTable,geocodedTableNew,copiedGeocodedTableNew]
        for j in i:
            if arcpy.Exists(j):
                arcpy.Delete_management(j)
        print(arcpy.GetMessages())
    except:
        print(arcpy.GetMessages())